In [ ]:
import os
os.environ['OUTDATED_IGNORE'] = '1'
import pandas as pd
import itertools
from scipy import stats
import pingouin as pg

# Root directory of the project
ROOT_DIR = os.getcwd()
if ROOT_DIR.endswith("notebooks"):
    # Go up one level to the repo root
    os.chdir(os.path.dirname(ROOT_DIR))

## Compute statistics for behavioral analysis Lab-Wue1 and save results

In [ ]:
df = pd.read_excel('source_data/fig_S7/behavior_wue1.xlsx')
l_param = ['freeze', 'distance']

l_results = []
for param in l_param:
    grp1_tr = df.loc[df.group == 1, param]
    grp2_tr = df.loc[df.group == 2, param]
    grp1_ret = df.loc[df.group == 3, param]
    grp2_ret = df.loc[df.group == 4, param]
    l_grps = [grp1_tr, grp2_tr, grp1_ret, grp2_ret]
       
    #Check for normal distribution with Shapiro-Wilk
    grp_norms = []
    for grp in l_grps:
        if grp.size > 3:
            grp_norms.append(stats.shapiro(grp)[1]>0.05)
    s_grp_norms = pd.Series(grp_norms)
    
    #Check for equality of variances with Levene
    levene = stats.levene(grp1_tr, grp2_tr, grp1_ret, grp2_ret, center = 'mean')[1]>0.05
    
    #Check if pre-requisites for parametric testing with One-Way ANOVA are fulfilled
    if (s_grp_norms.all() == True & levene == True):
        check_aov = True
    else:
        check_aov = False
    
    #Calculate One-Way ANOVA
    aov = pg.anova(data = df, between='group', dv=param, detailed=False)
    aov_dof1, aov_dof2, aov_fstat, aov_pval = aov['ddof1'][0], aov['ddof2'][0], aov['F'][0], aov['p-unc'][0]
    
    #Calculate Kruskal-Wallis ANOVA
    kwa_hstat, kwa_pval = stats.kruskal(grp1_tr, grp2_tr, grp1_ret, grp2_ret)
    
    #Calculate results for pairwise comparisons - parametric and non-parametric:
    l_t_res = []
    l_mwu_res = []
    for (grp1, grp2) in list(itertools.combinations([1, 2, 3, 4], 2)):
        ttest = pg.ttest(df.loc[df.group == grp1, param], df.loc[df.group == grp2, param], tail = 'two-sided', correction = 'auto')
        l_t_res.append((ttest['dof']['T-test'], ttest['T']['T-test'], ttest['p-val']['T-test']))
        u_stat, pval = stats.mannwhitneyu(df.loc[df.group == grp1, param],df.loc[df.group == grp2, param], alternative = 'two-sided')
        l_mwu_res.append((u_stat, pval))
    
    #Append results to list
    l_results.append((param, df.loc[df.group == 1, param].mean(), df.loc[df.group == 2, param].mean(),
                  df.loc[df.group == 3, param].mean(), df.loc[df.group == 4, param].mean(), 
                  check_aov, aov_dof1, aov_dof2, aov_fstat, aov_pval, 
                  l_t_res[0][0], l_t_res[0][1], l_t_res[0][2],
                  l_t_res[1][0], l_t_res[1][1], l_t_res[1][2],
                  l_t_res[2][0], l_t_res[2][1], l_t_res[2][2],
                  l_t_res[3][0], l_t_res[3][1], l_t_res[3][2],
                  l_t_res[4][0], l_t_res[4][1], l_t_res[4][2],
                  l_t_res[5][0], l_t_res[5][1], l_t_res[5][2],
                  kwa_hstat, kwa_pval,
                  l_mwu_res[0][0], l_mwu_res[0][1],
                  l_mwu_res[1][0], l_mwu_res[1][1],
                  l_mwu_res[2][0], l_mwu_res[2][1],
                  l_mwu_res[3][0], l_mwu_res[3][1],
                  l_mwu_res[4][0], l_mwu_res[4][1],
                  l_mwu_res[5][0], l_mwu_res[5][1]))

#Create results DataFrame
l_columns = ['Measure', 'Mean_grp1', 'Mean_grp2', 'Mean_grp3', 'Mean_grp4', 
              'ANOVA_ok', 'ddof1', 'ddof2', 'ANOVA_F_stat', 'ANOVA_p_value', 
              'T_test_dof_grp1_2', 'T_test_T_stat_grp1_2', 'T_test_pval_grp1_2',
              'T_test_dof_grp1_3', 'T_test_T_stat_grp1_3', 'T_test_pval_grp1_3',
              'T_test_dof_grp1_4', 'T_test_T_stat_grp1_4', 'T_test_pval_grp1_4',
              'T_test_dof_grp2_3', 'T_test_T_stat_grp2_3', 'T_test_pval_grp2_3',
              'T_test_dof_grp2_4', 'T_test_T_stat_grp2_4', 'T_test_pval_grp2_4',
              'T_test_dof_grp3_4', 'T_test_T_stat_grp3_4', 'T_test_pval_grp3_4',
              'Kruskal_Wallis_ANOVA_H_stat', 'Kruskal_Wallis_ANOVA_pval',
              'MWU_U_stat_grp1_2', 'MWU_pval_grp1_2',
              'MWU_U_stat_grp1_3', 'MWU_pval_grp1_3',
              'MWU_U_stat_grp1_4', 'MWU_pval_grp1_4',
              'MWU_U_stat_grp2_3', 'MWU_pval_grp2_3',
              'MWU_U_stat_grp2_4', 'MWU_pval_grp2_4',
              'MWU_U_stat_grp3_4', 'MWU_pval_grp3_4']    
df_results = pd.DataFrame(l_results, columns = l_columns)
df_results.set_index('Measure', inplace=True)
df_results
df_results.to_excel('source_data/fig_S7/stat_results_behavior_wue1.xlsx')